<a href="https://colab.research.google.com/github/Aytijha/Caramel_Popcorn/blob/main/NLP_Modelling.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np

import re, string
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import SnowballStemmer
from nltk.corpus import wordnet
from nltk.stem import WordNetLemmatizer
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
nltk.download('wordnet')

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import classification_report, f1_score, accuracy_score, confusion_matrix
from sklearn.metrics import roc_curve, auc, roc_auc_score

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import CountVectorizer

import gensim
from gensim.models import Word2Vec

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [2]:
import pandas as pd
nlp_df = pd.read_csv('NLP DF.csv')
nlp_df

,Unnamed: 0,Name of movie,Description,Genre
0,0,The Shawshank Redemption,Two imprisoned men bond over a number of years...,Drama
1,1,The Godfather,An organized crime dynasty's aging patriarch t...,Crime Drama
2,2,Soorarai Pottru,"Nedumaaran Rajangam ""Maara"" sets out to make t...",Drama
3,3,The Dark Knight,When the menace known as the Joker wreaks havo...,Action Crime Drama
4,4,The Godfather: Part II,The early life and career of Vito Corleone in ...,Crime Drama
...,...,...,...,...
995,995,Giant,Sprawling epic covering the life of a Texas ca...,Drama Western
996,996,From Here to Eternity,"In Hawaii in 1941, a private is cruelly punish...",Drama Romance War
997,997,Gilda,A small-time gambler hired to work in a Buenos...,Drama Film-Noir Romance
998,998,Lifeboat,Several survivors of a torpedoed merchant ship...,Drama War


In [3]:
nlp_df.drop('Unnamed: 0', inplace=True, axis=1)

In [4]:
nlp_df

,Name of movie,Description,Genre
0,The Shawshank Redemption,Two imprisoned men bond over a number of years...,Drama
1,The Godfather,An organized crime dynasty's aging patriarch t...,Crime Drama
2,Soorarai Pottru,"Nedumaaran Rajangam ""Maara"" sets out to make t...",Drama
3,The Dark Knight,When the menace known as the Joker wreaks havo...,Action Crime Drama
4,The Godfather: Part II,The early life and career of Vito Corleone in ...,Crime Drama
...,...,...,...
995,Giant,Sprawling epic covering the life of a Texas ca...,Drama Western
996,From Here to Eternity,"In Hawaii in 1941, a private is cruelly punish...",Drama Romance War
997,Gilda,A small-time gambler hired to work in a Buenos...,Drama Film-Noir Romance
998,Lifeboat,Several survivors of a torpedoed merchant ship...,Drama War


In [5]:
d = dict(nlp_df['Genre'].value_counts())
#d

In [6]:
nlp_df['Text'] = nlp_df['Name of movie'] + ' ' + nlp_df['Description']
nlp_df

,Name of movie,Description,Genre,Text
0,The Shawshank Redemption,Two imprisoned men bond over a number of years...,Drama,The Shawshank Redemption Two imprisoned men bo...
1,The Godfather,An organized crime dynasty's aging patriarch t...,Crime Drama,The Godfather An organized crime dynasty's agi...
2,Soorarai Pottru,"Nedumaaran Rajangam ""Maara"" sets out to make t...",Drama,"Soorarai Pottru Nedumaaran Rajangam ""Maara"" se..."
3,The Dark Knight,When the menace known as the Joker wreaks havo...,Action Crime Drama,The Dark Knight When the menace known as the J...
4,The Godfather: Part II,The early life and career of Vito Corleone in ...,Crime Drama,The Godfather: Part II The early life and care...
...,...,...,...,...
995,Giant,Sprawling epic covering the life of a Texas ca...,Drama Western,Giant Sprawling epic covering the life of a Te...
996,From Here to Eternity,"In Hawaii in 1941, a private is cruelly punish...",Drama Romance War,"From Here to Eternity In Hawaii in 1941, a pri..."
997,Gilda,A small-time gambler hired to work in a Buenos...,Drama Film-Noir Romance,Gilda A small-time gambler hired to work in a ...
998,Lifeboat,Several survivors of a torpedoed merchant ship...,Drama War,Lifeboat Several survivors of a torpedoed merc...


In [7]:
# converting text to lowercase, stripping and removing punctuations
def preprocess(text):
    text = text.lower() 
    text=text.strip()  
    text=re.compile('<.*?>').sub('', text) 
    text = re.compile('[%s]' % re.escape(string.punctuation)).sub(' ', text)  
    text = re.sub('\s+', ' ', text)  
    text = re.sub(r'\[[0-9]*\]',' ',text) 
    text=re.sub(r'[^\w\s]', '', str(text).lower().strip())
    text = re.sub(r'\d',' ',text) 
    text = re.sub(r'\s+',' ',text) 
    return text

In [8]:
# removing stopwords
def stopword(string):
    a= [i for i in string.split() if i not in stopwords.words('english')]
    return ' '.join(a)

In [9]:
# lemmatization
wl = WordNetLemmatizer()
 
# helper function to map NTLK position tags
def get_wordnet_pos(tag):
    if tag.startswith('J'):
        return wordnet.ADJ
    elif tag.startswith('V'):
        return wordnet.VERB
    elif tag.startswith('N'):
        return wordnet.NOUN
    elif tag.startswith('R'):
        return wordnet.ADV
    else:
        return wordnet.NOUN

# to tokenize the sentence
def lemmatizer(string):
    word_pos_tags = nltk.pos_tag(word_tokenize(string)) # Get position tags
    a=[wl.lemmatize(tag[0], get_wordnet_pos(tag[1])) for idx, tag in enumerate(word_pos_tags)] # Map the position tag and lemmatize the word/token
    return " ".join(a)

In [10]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [11]:
def finalpreprocess(string):
    return lemmatizer(stopword(preprocess(string)))
nlp_df['clean_text'] = nlp_df['Text'].apply(lambda x: finalpreprocess(x))
nlp_df

,Name of movie,Description,Genre,Text,clean_text
0,The Shawshank Redemption,Two imprisoned men bond over a number of years...,Drama,The Shawshank Redemption Two imprisoned men bo...,shawshank redemption two imprisoned men bond n...
1,The Godfather,An organized crime dynasty's aging patriarch t...,Crime Drama,The Godfather An organized crime dynasty's agi...,godfather organize crime dynasty age patriarch...
2,Soorarai Pottru,"Nedumaaran Rajangam ""Maara"" sets out to make t...",Drama,"Soorarai Pottru Nedumaaran Rajangam ""Maara"" se...",soorarai pottru nedumaaran rajangam maara set ...
3,The Dark Knight,When the menace known as the Joker wreaks havo...,Action Crime Drama,The Dark Knight When the menace known as the J...,dark knight menace know joker wreaks havoc cha...
4,The Godfather: Part II,The early life and career of Vito Corleone in ...,Crime Drama,The Godfather: Part II The early life and care...,godfather part ii early life career vito corle...
...,...,...,...,...,...
995,Giant,Sprawling epic covering the life of a Texas ca...,Drama Western,Giant Sprawling epic covering the life of a Te...,giant sprawl epic cover life texas cattle ranc...
996,From Here to Eternity,"In Hawaii in 1941, a private is cruelly punish...",Drama Romance War,"From Here to Eternity In Hawaii in 1941, a pri...",eternity hawaii private cruelly punish box uni...
997,Gilda,A small-time gambler hired to work in a Buenos...,Drama Film-Noir Romance,Gilda A small-time gambler hired to work in a ...,gilda small time gambler hire work buenos air ...
998,Lifeboat,Several survivors of a torpedoed merchant ship...,Drama War,Lifeboat Several survivors of a torpedoed merc...,lifeboat several survivor torpedo merchant shi...


In [12]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(nlp_df['clean_text'], nlp_df['Genre'], test_size=0.15)
print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)

(850,) (150,) (850,) (150,)


In [13]:
#Tf-Idf
tfidf_vectorizer = TfidfVectorizer(use_idf=True)
X_train_vectors_tfidf = tfidf_vectorizer.fit_transform(X_train) 
X_test_vectors_tfidf = tfidf_vectorizer.transform(X_test)

In [14]:
#FITTING THE CLASSIFICATION MODEL using Naive Bayes(tf-idf)
nb_tfidf = MultinomialNB()
nb_tfidf.fit(X_train_vectors_tfidf, y_train)  
#Predict y value for test dataset
y_predict = nb_tfidf.predict(X_test_vectors_tfidf)
y_prob = nb_tfidf.predict_proba(X_test_vectors_tfidf)[:,1]
#print(classification_report(y_test,y_predict))
#print('Confusion Matrix:',confusion_matrix(y_test, y_predict))

print(accuracy_score(y_test, y_predict))
 
'''fpr, tpr, thresholds = roc_curve(y_test, y_prob)
roc_auc = auc(fpr, tpr)
print('AUC:', roc_auc)'''

0.08666666666666667


"fpr, tpr, thresholds = roc_curve(y_test, y_prob)\nroc_auc = auc(fpr, tpr)\nprint('AUC:', roc_auc)"

In [15]:
#FITTING THE CLASSIFICATION MODEL using Logistic Regression(tf-idf)
lr_tfidf=LogisticRegression()
lr_tfidf.fit(X_train_vectors_tfidf, y_train)  
#Predict y value for test dataset
y_predict = lr_tfidf.predict(X_test_vectors_tfidf)
y_prob = lr_tfidf.predict_proba(X_test_vectors_tfidf)[:,1]
#print(classification_report(y_test,y_predict))
#print('Confusion Matrix:',confusion_matrix(y_test, y_predict))

print(accuracy_score(y_test, y_predict))

0.09333333333333334


In [16]:
nlp_df['Genre'].describe()

count      1000
unique      205
top       Drama
freq         88
Name: Genre, dtype: object

In [30]:
from collections import Counter
counter = Counter(nlp_df['Genre'].tolist())
top_40_genres = {i[0]: idx for idx, i in enumerate(counter.most_common(40))}
nlp_df = nlp_df[nlp_df['Genre'].map(lambda x: x in top_40_genres)]

In [31]:
description_list = nlp_df['clean_text'].tolist()
genre_list = [top_40_genres[i] for i in nlp_df['Genre'].tolist()]
genre_list = np.array(genre_list)
genre_list

array([ 0,  9,  0,  5,  9,  9, 11, 14,  9,  8, 10, 28,  0, 14,  1, 14, 13,
       23,  0,  6,  9, 33, 11,  3,  4,  6, 23, 14,  5, 29,  6, 30, 14,  0,
        5, 24,  1,  0, 21, 34,  3,  8,  0,  6,  8, 28, 17, 14, 10,  7,  2,
       33, 22, 21, 17,  9,  4, 15,  4, 11,  3,  5,  5,  6,  0,  0, 17,  0,
       31, 35,  0,  0,  1,  7,  8,  6, 11, 10,  8,  9, 23, 12,  9, 36,  3,
        0,  0, 37, 34, 29,  4,  7,  0,  4, 25, 11, 26,  5,  0,  2, 32, 21,
        7,  0,  4, 13,  0,  7,  0,  8, 33, 18,  4,  6,  6,  9, 12, 26,  9,
        8, 19, 22, 11,  1,  0,  4,  0, 25,  1,  6, 33, 12,  4,  0,  5, 16,
        7,  0, 22,  8,  7,  3,  8, 28, 28, 13, 24, 21,  4, 10,  5, 23,  0,
       17, 36,  6,  3,  0, 12, 38,  8,  1,  2,  4,  3,  5,  7, 13, 16,  7,
       22,  0, 27,  2,  1,  0,  6,  9,  1,  1, 32, 12, 10, 13, 27,  2, 39,
       11,  0,  0, 18, 37, 15, 11, 38,  0, 18, 12,  0,  4,  9, 16,  8, 11,
       11,  0,  8,  9,  0,  6,  6,  5, 27,  3, 32,  2, 11, 31, 11,  8, 31,
        1, 19, 23,  2, 36

In [32]:
count_vect = CountVectorizer()
x_train_counts = count_vect.fit_transform(description_list)

In [33]:
tfidf_transformer = TfidfTransformer()
x_train_tfidf = tfidf_transformer.fit_transform(x_train_counts)

In [34]:
train_x, test_x, train_y, test_y = train_test_split(x_train_tfidf, genre_list, test_size=0.15)

In [35]:
clf = MultinomialNB().fit(train_x, train_y)
y_score = clf.predict(train_x)

In [36]:
n_right = 0
for i in range(len(y_score)):
    if y_score[i] == train_y[i]:
        n_right += 1

print("Accuracy: %.2f%%" % ((n_right/float(len(train_y)) * 100)))

Accuracy: 17.49%


In [37]:
from sklearn.svm import SVC
clf1 = SVC().fit(train_x, train_y)

In [35]:
?SVC()

In [38]:
y_score = clf1.predict(train_x)
n_right = 0
for i in range(len(y_score)):
    if y_score[i] == train_y[i]:
        n_right += 1

print("Accuracy: %.2f%%" % ((n_right/float(len(train_y)) * 100)))

Accuracy: 73.22%


In [39]:
type(train_y)

numpy.ndarray

In [40]:
df_temp = pd.DataFrame()
df_temp['Predict'] = y_score.tolist()
df_temp['Actual'] = train_y.tolist()
df_temp

,Predict,Actual
0,4,4
1,0,0
2,0,20
3,1,1
4,15,15
...,...,...
544,1,1
545,0,28
546,7,7
547,0,29


In [28]:
nlp_df['Genre'].describe()

count       697
unique       50
top       Drama
freq         88
Name: Genre, dtype: object

In [29]:
l1 = dict(nlp_df['Genre'].value_counts())
l1

{'Action Adventure Comedy': 10,
 'Action Adventure Drama': 13,
 'Action Adventure Fantasy': 8,
 'Action Adventure Sci-Fi': 21,
 'Action Adventure Thriller': 5,
 'Action Biography Drama': 7,
 'Action Crime Drama': 29,
 'Action Crime Thriller': 10,
 'Adventure Biography Drama': 6,
 'Adventure Comedy Drama': 8,
 'Adventure Drama Fantasy': 5,
 'Adventure Family Fantasy': 6,
 'Animation Action Adventure': 12,
 'Animation Adventure Comedy': 29,
 'Animation Adventure Drama': 8,
 'Animation Adventure Family': 6,
 'Animation Drama Fantasy': 5,
 'Biography Comedy Drama': 7,
 'Biography Crime Drama': 14,
 'Biography Drama': 12,
 'Biography Drama History': 28,
 'Biography Drama Music': 7,
 'Biography Drama Sport': 8,
 'Comedy': 13,
 'Comedy Crime': 6,
 'Comedy Crime Drama': 15,
 'Comedy Drama': 36,
 'Comedy Drama Family': 6,
 'Comedy Drama Fantasy': 5,
 'Comedy Drama Romance': 31,
 'Comedy Romance': 7,
 'Crime Drama': 26,
 'Crime Drama Film-Noir': 5,
 'Crime Drama Mystery': 30,
 'Crime Drama Roman